In [19]:
import pandas as pd

In [20]:
new_albums = pd.read_csv("new_albums.csv")
top_200 = pd.read_csv("top_200_table.csv",sep="\t",index_col = 0)
del new_albums["Unnamed: 0"]

In [21]:
new_albums.head()

,albums,artists
0,Trident Death Rattle EP,Twilight
1,DOVE,Belly
2,Nothing Matters,Benjamin Lazar Davis
3,Songs in the Key of Funk,Big Sam’s Funky Nation
4,Love in Wartime,Birds of Chicago


In [22]:
top_200.head()

,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,...,2018-04-18,2018-04-19,2018-04-20,2018-04-21,2018-04-22,2018-04-23,2018-04-24,2018-04-25,2018-04-26,2018-04-27
0,"1,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","1,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","1,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","1,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","1,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","1,""Shape of You"",""Ed Sheeran"",1517787","1,""Shape of You"",""Ed Sheeran"",1437473","1,""Shape of You"",""Ed Sheeran"",1346146","1,""Shape of You"",""Ed Sheeran"",1532772","1,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...",...,"1,""Nice For What"",Drake,2784404","1,""Nice For What"",Drake,2704756","1,KOD,""J. Cole"",4233070","1,KOD,""J. Cole"",2537308","1,""Nice For What"",Drake,2232935","1,""Nice For What"",Drake,2494495","1,""Nice For What"",Drake,2545158","1,""Nice For What"",Drake,2369585","1,""Nice For What"",Drake,2351522","1,Paranoid,""Post Malone"",3694438"
1,"2,""Fake Love"",Drake,1180074","2,""Fake Love"",Drake,1076575","2,""Fake Love"",Drake,1171625","2,""Fake Love"",Drake,1179471","2,""Fake Love"",Drake,1179992","2,""Castle on the Hill"",""Ed Sheeran"",1472037","2,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","2,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","2,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","2,""Shape of You"",""Ed Sheeran"",1512685",...,"2,""God's Plan"",Drake,1801908","2,""God's Plan"",Drake,1787101","2,Photograph,""J. Cole"",3561388","2,""Nice For What"",Drake,2505932","2,KOD,""J. Cole"",1997552","2,KOD,""J. Cole"",2314828","2,KOD,""J. Cole"",2180639","2,KOD,""J. Cole"",1928183","2,KOD,""J. Cole"",1764366","2,""Better Now"",""Post Malone"",3467168"
2,"3,Starboy,""The Weeknd"",1064351","3,Starboy,""The Weeknd"",963602","3,Starboy,""The Weeknd"",1064931","3,Starboy,""The Weeknd"",1086272","3,Starboy,""The Weeknd"",1068015","3,""Bad and Boujee (feat. Lil Uzi Vert)"",Migos,...","3,""Fake Love"",Drake,1180717","3,""Fake Love"",Drake,1123166","3,""Fake Love"",Drake,1157963","3,""Fake Love"",Drake,1181414",...,"3,""I Like It"",""Cardi B"",1629604","3,""Psycho (feat. Ty Dolla $ign)"",""Post Malone""...","3,ATM,""J. Cole"",3408026","3,ATM,""J. Cole"",2223642","3,ATM,""J. Cole"",1771037","3,ATM,""J. Cole"",2065871","3,ATM,""J. Cole"",1805005","3,""Kevin’s Heart"",""J. Cole"",1623076","3,""God's Plan"",Drake,1611225","3,""Spoil My Night"",""Post Malone"",3318588"
3,"4,Closer,""The Chainsmokers"",1010492","4,Closer,""The Chainsmokers"",782423","4,Closer,""The Chainsmokers"",857394","4,Closer,""The Chainsmokers"",856349","4,Closer,""The Chainsmokers"",860364","4,""Fake Love"",Drake,1217137","4,""Castle on the Hill"",""Ed Sheeran"",1127503","4,Starboy,""The Weeknd"",933303","4,""Castle on the Hill"",""Ed Sheeran"",1016935","4,Starboy,""The Weeknd"",1022585",...,"4,""Psycho (feat. Ty Dolla $ign)"",""Post Malone""...","4,""I Like It"",""Cardi B"",1582355","4,Motiv8,""J. Cole"",3357778","4,Motiv8,""J. Cole"",2083997","4,""God's Plan"",Drake,1602919","4,Motiv8,""J. Cole"",1847631","4,""Kevin’s Heart"",""J. Cole"",1627749","4,""God's Plan"",Drake,1595052","4,""Kevin’s Heart"",""J. Cole"",1543022","4,Psycho,""Post Malone"",3171834"
4,"5,""Black Beatles"",""Rae Sremmurd"",874289","5,""Black Beatles"",""Rae Sremmurd"",733662","5,""Black Beatles"",""Rae Sremmurd"",769249","5,""I Don’t Wanna Live Forever (Fifty Shades Da...","5,""I Don’t Wanna Live Forever (Fifty Shades Da...","5,Starboy,""The Weeknd"",1083653","5,Starboy,""The Weeknd"",1002482","5,""Castle on the Hill"",""Ed Sheeran"",925352","5,Starboy,""The Weeknd"",990020","5,""Castle on the Hill"",""Ed Sheeran"",942521",...,"5,""Look Alive (feat. Drake)"",""BlocBoy JB"",1493428","5,""Look Alive (feat. Drake)"",""BlocBoy JB"",1482167","5,""Kevin’s Heart"",""J. Cole"",3200312","5,""Kevin’s Heart"",""J. Cole"",1994055","5,Motiv8,""J. Cole"",1598278","5,""Kevin

In [24]:
print(top_200["2018-03-05"])

0                           1,"God's Plan",Drake,2818168
1      2,"Psycho (feat. Ty Dolla $ign)","Post Malone"...
2      3,"Look Alive (feat. Drake)","BlocBoy JB",1540544
3      4,"All The Stars (with SZA)","Kendrick Lamar",...
4                            5,SAD!,XXXTENTACION,1234348
5                                   6,Mine,Bazzi,1142761
6                             7,"Stir Fry",Migos,1120026
7                            8,"The Middle",Zedd,1072158
8      9,"Pray For Me (with Kendrick Lamar)","The Wee...
9      10,"King's Dead (with Kendrick Lamar, Future &...
10           11,"Love Lies (with Normani)",Khalid,980852
11                12,"I Fall Apart","Post Malone",905064
12                      13,rockstar,"Post Malone",889474
13     14,"Ric Flair Drip (& Metro Boomin)",Offset,88...
14                              15,Everyday,Logic,774229
15                          16,FRIENDS,Marshmello,729589
16     17,"Meant to Be (feat. Florida Georgia Line)",...
17                             